<a href="https://colab.research.google.com/github/descartesmbogning/Marburg-virus-research/blob/main/2_last_authors_countries_groupby_worlbank_income.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re
from functools import reduce
import seaborn as sns
import missingno as msno
%matplotlib inline

import matplotlib.pyplot as plt
from scipy.stats import norm


In [ ]:
# Set the figure size - handy for larger output
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = [10, 6]
# Set up with a higher resolution screen (useful on Mac)
%config InlineBackend.figure_format = 'retina'

## import data

In [ ]:
data = pd.read_csv('D:\MARBURG VIRUS DISEASES\DATA_PREPROCESSING/countries_last_author.csv').drop(['Unnamed: 0'], axis=1)

data2 = data.rename(columns={"UT (Unique WOS ID)": "wos_ID"})
data2['wos_ID'] = data2['wos_ID'].str.lower()
data2

,wos_ID,Author Full Names,Addresses,last_author,Australia,Bangladesh,Belgium,United States,Canada,Colombia,...,Slovakia,South Africa,Spain,Sweden,Switzerland,Tanzania,Thailand,Turkey,Uganda,Zambia
0,wos:a1973q658600007,"MARTINI, GA","UNIV MARBURG,MARBURG,WEST GERMANY","MARTINI, GA",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,wos:a1974t169200002,"MONATH, TP","WORLD HLTH ORG,CTR DIS CONTROL,REG REFER CTR A...","MONATH, TP",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,wos:a1975ax37800009,"GEAR, JSS; CASSEL, GA; GEAR, AJ; TRAPPLER, B; ...","JOHANNESBURG GEN HOSP,JOHANNESBURG,SOUTH AFRIC...","GEAR, JHS",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,wos:a1977de18800006,"KUMING, BS; KOKORIS, N","JOHANNESBURG GEN HOSP,DEPT OPHTHALMOL,JOHANNES...","KOKORIS, N",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,wos:a1977cx78300009,"PATTYN, S; JACOB, W; VANDERGROEN, G; PIOT, P; ...","UNIV ANTWERP, ANTWERP, BELGIUM; INST TROP MED,...","COURTEILLE, G",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
871,wos:000398598700003,"Rojek, Amanda M.; Horby, Peter W.","[Rojek, Amanda M.; Horby, Peter W.] Univ Oxfor...","Horby, Peter W.",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
872,wos:000309742800063,"Paweska, Janusz T.; van Vuren, Petrus Jansen; ...","[Paweska, Janusz T.; van Vuren, Petrus Jansen;...","Kemp, Alan",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
873,wos:000460640400002,"Evans, Tierra Smiley; Tutaryebwa, Leonard; Gil...","[Evans, Tierra Smiley; Gilardi, Kirsten V.; Ma...","Johnson, Christine K.",NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
874,wos:000460640400054,"Selvaraj, Saranya A.; Lee, Karen E.; Harrell, ...","[Selvaraj, Saranya A.] Univ Maryland, Med Ctr,...","Allegranzi, Benedetta",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
data2.columns

Index(['wos_ID', 'Author Full Names', 'Addresses', 'last_author', 'Australia',
       'Bangladesh', 'Belgium', 'United States', 'Canada', 'Colombia',
       'Czech Republic', 'Denmark', 'Egypt', 'United Kingdom', 'Finland',
       'France', 'Gabon', 'Germany', 'Ghana', 'Greece', 'Guinea', 'India',
       'Indonesia', 'Italy', 'Japan', 'Jordan', 'Morocco', 'Netherlands',
       'New Zealand', 'Nigeria', 'Norway', 'Pakistan', 'Palestine', 'China',
       'Peru', 'Poland', 'Congo Republic', 'Russia', 'Saudi Arabia', 'Serbia',
       'Sierra Leone', 'Singapore', 'Slovakia', 'South Africa', 'Spain',
       'Sweden', 'Switzerland', 'Tanzania', 'Thailand', 'Turkey', 'Uganda',
       'Zambia'],
      dtype='object')

('Morocco',)

## list of countries

In [ ]:
data2.columns#.to_list()#[:20]

Index(['wos_ID', 'Author Full Names', 'Addresses', 'last_author', 'Australia',
       'Bangladesh', 'Belgium', 'United States', 'Canada', 'Colombia',
       'Czech Republic', 'Denmark', 'Egypt', 'United Kingdom', 'Finland',
       'France', 'Gabon', 'Germany', 'Ghana', 'Greece', 'Guinea', 'India',
       'Indonesia', 'Italy', 'Japan', 'Jordan', 'Morocco', 'Netherlands',
       'New Zealand', 'Nigeria', 'Norway', 'Pakistan', 'Palestine', 'China',
       'Peru', 'Poland', 'Congo Republic', 'Russia', 'Saudi Arabia', 'Serbia',
       'Sierra Leone', 'Singapore', 'Slovakia', 'South Africa', 'Spain',
       'Sweden', 'Switzerland', 'Tanzania', 'Thailand', 'Turkey', 'Uganda',
       'Zambia'],
      dtype='object')

In [ ]:
list_african_countries =  [  'Ghana',  'Central African Republic',
 'Uganda', "Cote d'Ivoire",   'Congo Republic',
 'Tanzania',  'Zambia',   'Guinea', 'Senegal', 'Egypt',  'Zimbabwe',
 'DR Congo',  'Nigeria',       'Malawi', 'Morocco',
 'Mali', 'South Africa',        'Angola',
  'Sierra Leone',    'Burkina Faso', 'Gabon']

In [ ]:
len(list_african_countries)

21

In [ ]:
north_africa = ['Morocco','Egypt']

In [ ]:
eastern_africa = [ 'Uganda', 'Tanzania']

In [ ]:
Southern_africa = ['South Africa','Zambia','Malawi','Zimbabwe','Angola']

In [ ]:
Western_africa = ['Ghana',"Cote d'Ivoire",'Guinea','Senegal', 'Sierra Leone',
                  'Burkina Faso','Mali','Nigeria']

In [ ]:
Central_africa = ['Congo Republic','Central African Republic',
                  'DR Congo','Gabon']

In [ ]:
non_african_countries = ['United States','Australia','Belgium','Brazil', 'Canada', 'Czech Republic', 'Denmark',
 'United Kingdom', 'Germany','France', 'Hungary','India', 'Israel','Italy', 'Japan','Lebanon',
'Netherlands','New Zealand','Norway','Serbia','Singapore','Sweden','Switzerland',
'Russia','Afghanistan','Austria','Bulgaria','Finland','China', 'Indonesia', 'Iran', 'Iraq',
  'Poland', 'Portugal',  'Saudi Arabia', 'Spain','Ukraine', 'Venezuela', 'Greece',
 'Bangladesh', 'Malaysia', 'Pakistan',   'Thailand',  'Colombia',
   'Jordan',    'United Arab Emirates',
  'Peru',     'Argentina', 'South Korea',
 'Nepal', 'Philippines', 'Belize',
  'Costa Rica', 'Honduras',  'Ecuador',    'Panama',
        'Vietnam', 'Croatia',
  'Slovakia',
     'Kazakhstan',
     'Palestine',

   'Turkey']

In [ ]:
#https://datahelpdesk.worldbank.org/knowledgebase/articles/906519-world-bank-country-and-lending-groups

# BY REGION

In [ ]:
wb_east_asia_and_pacific = ['South Korea','Philippines','Australia',  'Singapore',
                         'Malaysia','China',
                         'Thailand','Indonesia',
                         'Japan','New Zealand','Vietnam']
#Kiribati  French Polynesia Guam Nauru Tuvalu Palau  Northern Mariana Islands

In [ ]:
wb_europe_and_central_asia = ['Norway','Greece','Poland','Portugal','Austria','Hungary',
                           'Russia','Belgium','Serbia',
                           'Italy','Slovakia','Bulgaria','Kazakhstan','Spain',
                           'Croatia','Sweden','Switzerland','Czech Republic',
                           'Denmark','Turkey',
                           'Ukraine','Finland','United Kingdom','France',
                           'Netherlands','Germany']
# Gibraltar ,Greenland   Isle of Man  Channel Islands    Faroe Islands

In [ ]:
wb_latin_america_and_the_caribbean = ['Argentina','Peru',
                                   'Ecuador','Belize',
                                    'Brazil',
                                   'Honduras','Colombia',
                                   'Costa Rica','Venezuela','Panama']
# Puerto Rico  	Sint Maarten (Dutch part) 	St. Martin (French part) 	Virgin Islands (U.S.)
# British Virgin Islands  Cayman Islands  Turks and Caicos Islands

In [ ]:
wb_middle_east_and_north_africa = ['Jordan','Saudi Arabia','Lebanon','Egypt',
                                'Iran','United Arab Emirates','Iraq','Morocco','Israel']
# 	West Bank and Gaza

In [ ]:
wb_middle_east_and_north_africa_africa = ['Morocco','Egypt']
# 	West Bank and Gaza

In [ ]:
wb_middle_east_and_north_africa_others = ['Jordan','Saudi Arabia','Lebanon','Iran',
                                       'United Arab Emirates','Iraq','Israel']


In [ ]:
wb_north_america = ['Canada','United States']

In [ ]:
wb_south_asia = ['Afghanistan','India','Pakistan','Bangladesh','Nepal']

In [ ]:
wb_sub_saharan_africa = ['Angola','Gabon', 'Nigeria', 'Burkina Faso','Ghana',
                      'Guinea','Senegal', 'Central African Republic','Congo Republic',
                      'Sierra Leone','South Africa',
                      'DR Congo','Malawi','Mali','Tanzania',
                      "Cote d'Ivoire",'Uganda','Zambia',
                      'Zimbabwe']

In [ ]:
# BY INCOME

In [ ]:
wb_low_income_economies = ['Afghanistan','Burkina Faso','Central African Republic',
                        'Malawi','DR Congo','Mali','Uganda','Congo Republic',
                        'Zambia', 'Guinea','Sierra Leone']

In [ ]:
wb_low_income_economies_africa = ['Burkina Faso',
                        'Malawi','DR Congo','Mali','Uganda',
                        'Zambia', 'Guinea','Sierra Leone']

In [ ]:
wb_low_income_economies_others = ['Afghanistan']

In [ ]:
wb_lower_middle_income_economies = ['Angola', 'India', 'Philippines', 'Indonesia','Bangladesh', 'Iran',
                                  'Senegal','Morocco',
                                  'Tanzania', 'Lebanon',

                                 'Ukraine',"Cote d'Ivoire",
                                 'Egypt','Vietnam','Nepal','Zimbabwe',
                                 'Ghana','Nigeria','Pakistan','Honduras']
#Kiribati 	West Bank and Gaza

In [ ]:
wb_lower_middle_income_economies_africa = ['Angola', 'Senegal',
                                        'Tanzania','Morocco',
                                        "Cote d'Ivoire",'Egypt','Zimbabwe','Ghana','Nigeria']
# 	West Bank and Gaza

In [ ]:
wb_lower_middle_income_economies_others = ['India','Philippines','Indonesia','Bangladesh','Iran',
                                        'Lebanon',
                                        'Ukraine',
                                        'Vietnam','Nepal','Pakistan','Honduras']
#Kiribati 	West Bank and Gaza

In [ ]:
wb_upper_middle_income_economies = ['Gabon','Argentina',
                                 'Peru','Russia','Belize','Iraq','Serbia',
                                 'South Africa','Jordan','Brazil',
                                 'Kazakhstan','Bulgaria','China',
                                 'Thailand','Colombia','Malaysia','Costa Rica','Turkey',

                                 'Ecuador']
# 'Tuvalu' 	Palau

In [ ]:
wb_upper_middle_income_economies_africa = ['Gabon','South Africa']

In [ ]:
wb_upper_middle_income_economies_others = ['Argentina',
                                        'Peru','Russia','Belize','Iraq','Serbia',
                                        'Jordan','Brazil','Kazakhstan',
                                        'Bulgaria','China','Thailand',
                                        'Colombia','Malaysia','Costa Rica','Turkey',
                                         'Ecuador']
# 'Tuvalu' 	Palau

In [ ]:
wb_high_income_economies = [ 'Greece','Poland', 'Portugal','Australia','Austria','Hungary',
                         'Saudi Arabia',
                         'Belgium','Israel','Singapore','Italy','Japan','Slovakia','South Korea',
                         'Canada','Spain',
                         'Sweden','Croatia','Switzerland',
                         'Czech Republic','Denmark','United Arab Emirates','Netherlands','United Kingdom',
                         'United States','Finland','New Zealand','France','Norway','Germany',
                         'Panama']
# Channel Islands 	Gibraltar Aruba 	Northern Mariana Islands 	Virgin Islands (U.S.)	Nauru 	Sint Maarten (Dutch part) 	Isle of Man 	British Virgin Islands
# Cayman Islands 	French Polynesia  Faroe Islands  	Turks and Caicos Islands 	St. Martin (French part)	'Guam' 	Puerto Rico, Greenland,

In [ ]:
#wb_high_income_economies_africa = []

In [ ]:
wb_high_income_economies_others = [ 'Greece','Poland', 'Portugal','Australia','Austria','Hungary',
                         'Saudi Arabia',
                         'Belgium','Israel','Singapore','Italy','Japan','Slovakia','South Korea',
                         'Canada','Spain',
                         'Sweden','Croatia','Switzerland',
                         'Czech Republic','Denmark','United Arab Emirates','Netherlands','United Kingdom',
                         'United States','Finland','New Zealand','France','Norway','Germany',
                         'Panama']
# Channel Islands 	Gibraltar Aruba 	Northern Mariana Islands 	Virgin Islands (U.S.)	Nauru 	Sint Maarten (Dutch part) 	Isle of Man 	British Virgin Islands
# Cayman Islands 	French Polynesia  Faroe Islands  	Turks and Caicos Islands 	St. Martin (French part)	'Guam' 	Puerto Rico, Greenland,

In [ ]:
# BY LENDING

### count group

In [ ]:
groups_values = [north_africa, eastern_africa, Southern_africa, Western_africa, Central_africa, non_african_countries,
                 wb_east_asia_and_pacific, wb_europe_and_central_asia, wb_latin_america_and_the_caribbean, wb_middle_east_and_north_africa, wb_middle_east_and_north_africa_africa, wb_middle_east_and_north_africa_others, wb_north_america, wb_south_asia, wb_sub_saharan_africa,
                 wb_low_income_economies, wb_low_income_economies_africa, wb_low_income_economies_others, wb_lower_middle_income_economies, wb_lower_middle_income_economies_africa, wb_lower_middle_income_economies_others, wb_upper_middle_income_economies, wb_upper_middle_income_economies_africa, wb_upper_middle_income_economies_others, wb_high_income_economies,  wb_high_income_economies_others
                 ]

In [ ]:
#groups_values

In [ ]:
groups_keys = ['north_africa', 'eastern_africa', 'Southern_africa', 'Western_africa', 'Central_africa', 'non_african_countries',

               'wb_east_asia_and_pacific', 'wb_europe_and_central_asia', 'wb_latin_america_and_the_caribbean', 'wb_middle_east_and_north_africa', 'wb_middle_east_and_north_africa_africa', 'wb_middle_east_and_north_africa_others', 'wb_north_america', 'wb_south_asia', 'wb_sub_saharan_africa',
                 'wb_low_income_economies', 'wb_low_income_economies_africa', 'wb_low_income_economies_others', 'wb_lower_middle_income_economies', 'wb_lower_middle_income_economies_africa', 'wb_lower_middle_income_economies_others', 'wb_upper_middle_income_economies', 'wb_upper_middle_income_economies_africa', 'wb_upper_middle_income_economies_others', 'wb_high_income_economies',  'wb_high_income_economies_others'
                  ]

In [ ]:
groups_keys

['north_africa',
 'eastern_africa',
 'Southern_africa',
 'Western_africa',
 'Central_africa',
 'non_african_countries',
 'wb_east_asia_and_pacific',
 'wb_europe_and_central_asia',
 'wb_latin_america_and_the_caribbean',
 'wb_middle_east_and_north_africa',
 'wb_middle_east_and_north_africa_africa',
 'wb_middle_east_and_north_africa_others',
 'wb_north_america',
 'wb_south_asia',
 'wb_sub_saharan_africa',
 'wb_low_income_economies',
 'wb_low_income_economies_africa',
 'wb_low_income_economies_others',
 'wb_lower_middle_income_economies',
 'wb_lower_middle_income_economies_africa',
 'wb_lower_middle_income_economies_others',
 'wb_upper_middle_income_economies',
 'wb_upper_middle_income_economies_africa',
 'wb_upper_middle_income_economies_others',
 'wb_high_income_economies',
 'wb_high_income_economies_others']

In [ ]:
# using zip()
# to convert lists to dictionary
groups_keys_values = dict(zip(groups_keys, groups_values))

In [ ]:
#groups_keys_values

In [ ]:
groups_keys_values['north_africa']

['Morocco', 'Egypt']

In [ ]:
# for group_key in groups_keys:
#     data2[group_key] = data2[groups_keys_values[group_key]].count(axis=1)

In [ ]:
groups_keys_values['eastern_africa']

['Uganda', 'Tanzania']

In [ ]:
# # create a dictionary mapping group keys to a list of valid column names
# # groups_keys_values = {}
# for group_key in groups_keys:
#     valid_columns = [col for col in groups_keys_values[group_key] if col in data2.columns]
#     groups_keys_values[group_key] = valid_columns
# groups_keys_values

In [ ]:
def merge_count_columns(data2, groups_keys):
    # create a dictionary mapping group keys to a list of valid column names
#     groups_keys_values = {}
    for group_key in groups_keys:
        valid_columns = [col for col in groups_keys_values[group_key] if col in data2.columns]
        groups_keys_values[group_key] = valid_columns

    # iterate over group keys and count the values in data2
    for group_key in groups_keys:
        data2_cols = groups_keys_values[group_key]
        if len(data2_cols) == 0:
            print(f"No valid columns found for group_key: {group_key}. Skipping...")
            continue
        data2[group_key] = data2[data2_cols].count(axis=1)

    # merge data1 and data2 on the group keys
#     merged_data = pd.merge(data1, data2, on=groups_keys)

    return data2

In [ ]:
data_group = merge_count_columns(data2, groups_keys)
data_group.head(2)

No valid columns found for group_key: wb_low_income_economies_others. Skipping...


,wos_ID,Author Full Names,Addresses,last_author,Australia,Bangladesh,Belgium,United States,Canada,Colombia,...,wb_low_income_economies,wb_low_income_economies_africa,wb_lower_middle_income_economies,wb_lower_middle_income_economies_africa,wb_lower_middle_income_economies_others,wb_upper_middle_income_economies,wb_upper_middle_income_economies_africa,wb_upper_middle_income_economies_others,wb_high_income_economies,wb_high_income_economies_others
0,wos:a1973q658600007,"MARTINI, GA","UNIV MARBURG,MARBURG,WEST GERMANY","MARTINI, GA",NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
1,wos:a1974t169200002,"MONATH, TP","WORLD HLTH ORG,CTR DIS CONTROL,REG REFER CTR A...","MONATH, TP",NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
data_group.shape

(876, 77)

In [ ]:
data_group = data_group.replace({0: np.nan})

In [ ]:
#save data
data_group.to_pickle('D:\MARBURG VIRUS DISEASES\DATA_PREPROCESSING/data_concat_countries_and_wb_groups_income_last_authors.pkl')
data_group.to_csv('D:\MARBURG VIRUS DISEASES\DATA_PREPROCESSING/data_concat_countries_and_wb_groups_income_last_authors.csv')
data_group

,wos_ID,Author Full Names,Addresses,last_author,Australia,Bangladesh,Belgium,United States,Canada,Colombia,...,wb_low_income_economies,wb_low_income_economies_africa,wb_lower_middle_income_economies,wb_lower_middle_income_economies_africa,wb_lower_middle_income_economies_others,wb_upper_middle_income_economies,wb_upper_middle_income_economies_africa,wb_upper_middle_income_economies_others,wb_high_income_economies,wb_high_income_economies_others
0,wos:a1973q658600007,"MARTINI, GA","UNIV MARBURG,MARBURG,WEST GERMANY","MARTINI, GA",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,wos:a1974t169200002,"MONATH, TP","WORLD HLTH ORG,CTR DIS CONTROL,REG REFER CTR A...","MONATH, TP",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,wos:a1975ax37800009,"GEAR, JSS; CASSEL, GA; GEAR, AJ; TRAPPLER, B; ...","JOHANNESBURG GEN HOSP,JOHANNESBURG,SOUTH AFRIC...","GEAR, JHS",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,wos:a1977de18800006,"KUMING, BS; KOKORIS, N","JOHANNESBURG GEN HOSP,DEPT OPHTHALMOL,JOHANNES...","KOKORIS, N",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,wos:a1977cx78300009,"PATTYN, S; JACOB, W; VANDERGROEN, G; PIOT, P; ...","UNIV ANTWERP, ANTWERP, BELGIUM; INST TROP MED,...","COURTEILLE, G",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
871,wos:000398598700003,"Rojek, Amanda M.; Horby, Peter W.","[Rojek, Amanda M.; Horby, Peter W.] Univ Oxfor...","Horby, Peter W.",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0
872,wos:000309742800063,"Paweska, Janusz T.; van Vuren, Petrus Jansen; ...","[Paweska, Janusz T.; van Vuren, Petrus Jansen;...","Kemp, Alan",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN
873,wos:000460640400002,"Evans, Tierra Smiley; Tutaryebwa, Leonard; Gil...","[Evans, Tierra Smiley; Gilardi, Kirsten V.; Ma...","Johnson, Christine K.",NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0
874,wos:000460640400054,"Selvaraj, Saranya A.; Lee, Karen E.; Harrell, ...","[Selvaraj, Saranya A.] Univ Maryland, Med Ctr,...","Allegranzi, Benedetta",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0
